In [1]:
import pandas as pd
import matplotlib.pyplot as plt

#### importação de dados ###

In [2]:
df = pd.read_csv('zap_imoveis.csv', index_col='id')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3944 entries, 0 to 3943
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   condominio   3944 non-null   float64
 1   anunciante   3944 non-null   object 
 2   num_suites   3944 non-null   float64
 3   num_vagas    3944 non-null   float64
 4   num_quartos  3944 non-null   float64
 5   area         3944 non-null   float64
 6   rua          2932 non-null   object 
 7   regiao       3944 non-null   object 
 8   descricao    3764 non-null   object 
 9   tipo_imovel  3944 non-null   object 
 10  localidade   3944 non-null   object 
 11  bairro       3944 non-null   object 
 12  preco        3944 non-null   int64  
dtypes: float64(5), int64(1), object(7)
memory usage: 431.4+ KB


#### limpeza da base

In [3]:
df = df.drop(['anunciante','descricao','localidade','rua','regiao','bairro'], axis = 1)
df = df.query("tipo_imovel == 'Apartamento' or tipo_imovel == 'Casa' or tipo_imovel == 'Casa de Condomínio'")
df.head().T

id,0,1,2,3,4
condominio,0.0,0.0,18.0,230.0,680.0
num_suites,2.0,0.0,0.0,0.0,1.0
num_vagas,2.0,0.0,1.0,1.0,1.0
num_quartos,3.0,0.0,2.0,2.0,3.0
area,150.0,360.0,45.0,55.0,147.0
tipo_imovel,Casa,Casa,Casa,Casa,Casa
preco,570000,200000,170000,250000,600000


In [4]:
df['tipo_imovel'].value_counts()


Apartamento           1623
Casa                  1502
Casa de Condomínio     150
Name: tipo_imovel, dtype: int64

## KNN

#### definição de dados de treino e teste

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.drop('tipo_imovel',axis=1), df['tipo_imovel'], test_size=0.3)

#### normalização dos dados

In [6]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()  
scaler.fit(X_train)
X_train = scaler.transform(X_train)  
X_test = scaler.transform(X_test)

#### Definição do número de vizinhos no classificador

In [7]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3)

#### aplicando dados de treino ao knn

In [8]:
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

#### aplicando knn aos dados de teste

In [9]:
resultado = knn.predict(X_test)

#### criando matriz de confusão

In [10]:
print (pd.crosstab(y_test,resultado, rownames=['Real'], colnames=['Predicted'], margins=True))

Predicted           Apartamento  Casa  Casa de Condomínio  All
Real                                                          
Apartamento                 384    86                   7  477
Casa                         69   371                  11  451
Casa de Condomínio           25    18                  12   55
All                         478   475                  30  983


#### Relatório da classificação KNN

In [11]:
from sklearn import metrics
print(metrics.classification_report(y_test,resultado))

                    precision    recall  f1-score   support

       Apartamento       0.80      0.81      0.80       477
              Casa       0.78      0.82      0.80       451
Casa de Condomínio       0.40      0.22      0.28        55

          accuracy                           0.78       983
         macro avg       0.66      0.62      0.63       983
      weighted avg       0.77      0.78      0.77       983



#### verificando ajustes para o modelo ###

In [12]:
from sklearn.model_selection import GridSearchCV


#### adicionando a quantidade de valores para k (número de vizinhos) a um dicionário ###

In [13]:
k_list = list(range(1,31))
parametros = dict(n_neighbors=k_list)


#### passando nosso modelo, os valores para k, dobras pra validação cruzada e o scoring para o objeto GridSearch ###

In [14]:
grid = GridSearchCV(knn, parametros, cv=5, scoring='accuracy')


#### treinando o objeto ###

In [15]:
grid.fit(df.drop('tipo_imovel',axis=1), df['tipo_imovel'])

GridSearchCV(cv=5, estimator=KNeighborsClassifier(n_neighbors=3),
             param_grid={'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                         13, 14, 15, 16, 17, 18, 19, 20, 21, 22,
                                         23, 24, 25, 26, 27, 28, 29, 30]},
             scoring='accuracy')

#### descobrindo o parâmetro com melhor acurácia ###

In [16]:
print("Melhor parâmetro {} com acurácia de {} ".format(grid.best_params_,grid.best_score_))

Melhor parâmetro {'n_neighbors': 3} com acurácia de 0.7282442748091602 


## Decision Tree


In [17]:
from sklearn.tree import DecisionTreeClassifier

#### verificação das formas dos dados

In [18]:
X_train.shape,X_test.shape

((2292, 6), (983, 6))

In [19]:
y_train.shape,y_test.shape


((2292,), (983,))

#### Instanciando classificador

In [20]:
clf = DecisionTreeClassifier(criterion="entropy")

#### Treinamentodo modelo

In [21]:
clf = clf.fit(X_train,y_train)

#### Verificação das features mais importantes

In [22]:
for feature,importancia in zip(df.columns,clf.feature_importances_):
    print("{}:{}".format(feature,importancia))


condominio:0.30462396308794426
num_suites:0.047918265128657156
num_vagas:0.07565817107836371
num_quartos:0.02686752050630142
area:0.2624099507446917
tipo_imovel:0.2825221294540418


#### Predição

In [23]:
resultado = clf.predict(X_test)
#resultado

#### Matriz de confusão

In [24]:
print (pd.crosstab(y_test,resultado, rownames=['Real'], colnames=['Predicted'], margins=True))

Predicted           Apartamento  Casa  Casa de Condomínio  All
Real                                                          
Apartamento                 383    72                  22  477
Casa                         70   367                  14  451
Casa de Condomínio           25    19                  11   55
All                         478   458                  47  983


#### Relatório da classificação Árvore de Decisão

In [25]:
print(metrics.classification_report(y_test,resultado))

                    precision    recall  f1-score   support

       Apartamento       0.80      0.80      0.80       477
              Casa       0.80      0.81      0.81       451
Casa de Condomínio       0.23      0.20      0.22        55

          accuracy                           0.77       983
         macro avg       0.61      0.61      0.61       983
      weighted avg       0.77      0.77      0.77       983

